In [2]:
# Call necessary libraries
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import date, datetime
import re
import math
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly as py
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_csv('https://query.data.world/s/gib6aa6n3tmtuvrqpbdlhcbgdg7tk4')


In [6]:
df.shape

(1586614, 13)

In [7]:
# Count of unique breweries => Integrity issues evidenced => Id is not considered for subsequent analysis
print( 'Unique breweries' )
print( 'By id:', df[ 'brewery_id' ].nunique() )
print( 'By name:', df[ 'brewery_name' ].nunique() )

Unique breweries
By id: 5840
By name: 5742


In [8]:
# Count of unique beers => Integrity issues evidenced => Id is not considered for subsequent analysis
print( 'Unique beers' )
print( 'By id:', df[ 'beer_beerid' ].nunique() )
print( 'By name:', df[ 'beer_name' ].nunique() )

Unique beers
By id: 66055
By name: 56857


In [9]:
# Reviews by user
df[ 'review_profilename' ].value_counts()

northyorksammy    5817
BuckeyeNation     4661
mikesgroove       4617
Thorpe429         3518
womencantsail     3497
                  ... 
Rikadyn              1
ruffbrew             1
JGS                  1
CplSleden            1
oldbeerfan           1
Name: review_profilename, Length: 33387, dtype: int64

In [10]:
# A beer subset removing review information is created 
grouped_beers_df = df[ [ 'beer_name', 'brewery_name', 'beer_style', 'beer_abv', 'review_overall' ] ].drop_duplicates()

In [11]:
# Count of beers with the same name but different brewery, style or AVB%
beers = grouped_beers_df.loc[ grouped_beers_df.duplicated( subset = [ 'beer_name' ], keep = False ) ].sort_values( by = 'beer_name'  ).shape

In [12]:
# Beers by style
grouped_beers_df[ 'beer_style' ].value_counts( dropna = False )

American IPA                       9960
American Pale Ale (APA)            8654
American Amber / Red Ale           6551
American Porter                    5799
American Double / Imperial IPA     4928
                                   ... 
Kvass                               116
Gose                                104
Bière de Champagne / Bière Brut      88
Sahti                                87
Faro                                 81
Name: beer_style, Length: 104, dtype: int64

In [19]:
# create a list of our conditions
conditions = [
    (grouped_beers_df['beer_abv'] <= 0.5),
    (grouped_beers_df['beer_abv'] >= 0.51) & (grouped_beers_df['beer_abv'] <= 3.299999),
    (grouped_beers_df['beer_abv'] >= 3.3) & (grouped_beers_df['beer_abv'] <= 5.99999),
    (grouped_beers_df['beer_abv'] >= 6.0)
    ]

# create a list of the values we want to assign for each condition
values = ['NA', 'Low', 'Reg', 'High']

# create a new column and use np.select to assign values to it using our lists as arguments
grouped_beers_df['beer_strength'] = np.select(conditions, values, default="unknown")

# remove "0" values


# display updated DataFrame
grouped_beers_df.head()

,beer_name,brewery_name,beer_style,beer_abv,review_overall,beer_strength
0,Sausa Weizen,Vecchio Birraio,Hefeweizen,5.0,1.5,Reg
1,Red Moon,Vecchio Birraio,English Strong Ale,6.2,3.0,High
2,Black Horse Black Beer,Vecchio Birraio,Foreign / Export Stout,6.5,3.0,High
3,Sausa Pils,Vecchio Birraio,German Pilsener,5.0,3.0,Reg
4,Cauldron DIPA,Caldera Brewing Company,American Double / Imperial IPA,7.7,4.0,High


In [20]:
grouped_beers_df.groupby(['beer_strength']).count()

,beer_name,brewery_name,beer_style,beer_abv,review_overall
beer_strength,,,,,
High,69733,69727,69733,69733,69733
Low,1071,1071,1071,1071,1071
NA,295,295,295,295,295
Reg,71648,71639,71648,71648,71648
unknown,33097,33097,33097,0,33097


In [22]:
grouped_beers_df.to_csv('beers.csv', index=False)